# Tasca base de dades NoSQL
**Descripció**
Comencem a familiaritzar-nos amb bases de dades NoSQL !!! Comencem amb uns quants exercicis bàsics

### Recursos
Recursos de l'aula i [https://docs.spring.io/spring-data/mongodb/docs/current/reference/html/#reference](https://docs.spring.io/spring-data/mongodb/docs/current/reference/html/#reference) .

## Nivell 1
### Exercici 1
Crea una base de dades NoSQL utilitzant MongoDB. Afegeix-li algunes dades d'exemple que et permetin comprovar que ets capaç de processar-ne la informació de manera bàsica.

In [53]:
import pymongo
from configparser import ConfigParser


import json
# import pandas as pd

# import seaborn as sns

In [2]:
cfg = ConfigParser()
cfg.read('../config/mongodb-atlas.cfg')

host = cfg.get('atlas','host')
db = cfg.get('atlas','db')
user = cfg.get('atlas','user')
passwd = cfg.get('atlas','passwd')

### Exercici 2
Connecta la base de dades NoSQL a Python utilitzant per exemple pymongo.

In [3]:

client = pymongo.MongoClient(f"mongodb+srv://{user}:{passwd}@{host}/{db}?retryWrites=true&w=majority")
db = client.test


In [14]:
cursor = client.list_databases()

for c in cursor:
    print(c['name'])

sample_airbnb
sample_analytics
sample_geospatial
sample_mflix
sample_restaurants
sample_supplies
sample_training
sample_weatherdata
admin
local


In [18]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

In [28]:
s_abnb = client.get_database(name='sample_restaurants')

In [29]:
s_abnb.list_collection_names()

['neighborhoods', 'restaurants']

In [40]:
search = s_abnb.get_collection('restaurants').find({'borough':'Manhattan','cuisine':'Delicatessen', })

for s in search:
    print(s)

In [45]:
search = s_abnb.get_collection('restaurants').find({'borough':'Manhattan','cuisine':'Delicatessen', })

search.collection.estimated_document_count()

25359

In [47]:
search.collection.count_documents(filter={})

25359

## Nivell 2
### Exercici 1
Carega algunes consultes senzilles a un Pandas Dataframe. 

## Nivell 3
### Exercici 1
Genera un resum estadístic de la informació que conté la base de dades.

In [50]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

In [52]:
client.get_database('sample_training').list_collection_names()

['posts', 'grades', 'companies', 'routes', 'trips', 'zips', 'inspections']